In [39]:
import pandapower as pp
import pandapower.networks as pn
import numpy as np
from datetime import datetime

# Loading the CASE33BW from Matpower
net = pn.case33bw()
# Removing the buses geo data
net.pop("bus_geodata")

### Buses

In [32]:
# Add the FV buses to the network at buses 25-33 (remember that it start counts the buses from 0 index)

# First create an index that fits all the new values after the row we want to add
# Add these new buses to the network
# Sort the new indexes and then reset it so it can create new indexes without flot numbers (with drop=true to remove the old index column) 
added_buses = np.linspace(24.1,24.9, 9)
for i in added_buses:
    pp.create_bus(net, 0.4, index = i, max_vm_pu = 1.08, min_vm_pu=0.92, zone=1.0)
net.bus = net.bus.sort_index().reset_index(drop=True)

# Correcting the buses names 
for i in range(25,42):
    net.bus.loc[i, "name"] = i


### Loads

In [33]:
# Add the first load (index = 0) to be equal to the matlab case
pp.create_load(net, 0, p_mw=0, q_mvar=0, index=-1, controllable = False, in_service = True, type=None)
net.load = net.load.sort_index().reset_index(drop=True)

# Changing the load at bus 24 (bus 25 in matlab) to active and reactive power = 0 
net.load.loc[24, ["p_mw", "q_mvar"]] = 0.0

# Add the FV load to the network at buses 25-33 (remember that it start counts the buses from 0 index)
# Same method used with buses
index = np.linspace(24.1, 24.9, 9)
added_buses = np.arange(25,34)
pd = np.array([0.0, 3.1, 2.1, 2.5, 2.6, 2.0, 3.1, 2.3, 2.6])
for i, j, k in zip(added_buses, pd, index):
    pp.create_load(net, i, p_mw=j/1000, index=k, q_mvar=0, controllable = False, in_service = True, type=None)
net.load = net.load.sort_index().reset_index(drop=True)

# Correcting the buses that the remaining loads are connected 
for i in range(34,42):
    net.load.loc[i, "bus"] = i

cols = ['bus',
 'p_mw',
 'q_mvar',
 'const_i_percent',
 'const_z_percent',
 'controllable',
 'in_service',
 'name',
 'scaling',
 'sn_mva',
 'type']
# net.load[cols]

### Lines

In [34]:
# Removing the lines 32-37 that not appears in the matlab file
net.line.drop(np.arange(32,37), axis=0, inplace=True)

# Changing the columns position for a better reading
# Used cols = df.columns.tolist() to get this line_columns list
line_columns = ['from_bus',
 'to_bus',
 'r_ohm_per_km',
 'x_ohm_per_km',
 'c_nf_per_km',
 'g_us_per_km',
 'in_service',
 'length_km',
 'max_i_ka',
 'max_loading_percent',
 'name',
 'parallel',
 'std_type',
 'type',
 'df']
net.line = net.line[line_columns]

# Adjusting the buses of index 24-31 and changing its "from and to" buses
from_bus = np.array([5, 34,	35,	36,	37,	38,	39,	40])
to_bus = np.array([34, 35, 36, 37, 38, 39, 40, 41])
for i in range(24,32):
    net.line.loc[i, ["from_bus", "to_bus"]] = from_bus[i-24], to_bus[i-24]

# Creating new lines
from_bus = np.array([24, 25, 26, 27, 28, 29, 30, 31, 32])
to_bus = np.array([25, 26, 27, 28, 29, 30, 31, 32, 33])
r = np.array([0.0000, 0.0176, 0.0320, 0.0208, 0.0544, 0.0208, 0.0448, 0.0672, 0.0224])
x = np.array([0.1024, 0.0040, 0.0072, 0.0047, 0.0123, 0.0047, 0.0101, 0.0152, 0.0051])

for i, j, k, z in zip(from_bus, to_bus, r, x):
    pp.create_line_from_parameters(net, from_bus=i, to_bus=j, r_ohm_per_km=k, x_ohm_per_km=z, length_km=1.0, c_nf_per_km=0.0, max_i_ka=99999.0, max_loading_percent=100.0, type="ol")

In [35]:
date = datetime.now().strftime('%d_%m-%H_%M')

# Saving the modified network to a xlsx file
pp.to_excel(net, f"case33bw_mod_{date}.xlsx")